<a href="https://colab.research.google.com/github/GenaroHacker/actionable_builder/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
#@title Set Up
!git clone https://github.com/GenaroHacker/write_imports.git
from write_imports.write_imports import write_imports

!git clone https://github.com/GenaroHacker/actionable_builder.git


In [2]:

# Delete this line to see the new imports
%%capture



#@title Import Statements
#Modules: ['write_imports', 'actionable_builder']



from write_imports.write_imports import write_imports

from write_imports.LearningPython.BasicExamples.intro_functions import print_hello_world

from actionable_builder.modules.add_example_data import add_example_data

from actionable_builder.modules.create_database import create_table
from actionable_builder.modules.create_database import insert_record
from actionable_builder.modules.create_database import create_empty_database
from actionable_builder.modules.create_database import check_or_insert_with_parent
from actionable_builder.modules.create_database import check_step_exists
from actionable_builder.modules.create_database import add_task
from actionable_builder.modules.create_database import add_placeholders
from actionable_builder.modules.create_database import initialize_database
from actionable_builder.modules.create_database import create_example_database
from actionable_builder.modules.create_database import download_database

from actionable_builder.modules.sql_core import insert_several_records
from actionable_builder.modules.sql_core import read_records
from actionable_builder.modules.sql_core import read_last_record
from actionable_builder.modules.sql_core import update_record
from actionable_builder.modules.sql_core import remove_record
from actionable_builder.modules.sql_core import run_command

from actionable_builder.modules.pick_zipfian import pick_zipfian





print(write_imports([]))





In [3]:
# @title Main Execution

# Main Execution
if __name__ == "__main__":
    initialize_database()

    # Example task input
    task = {
        "place": "gym",
        "context": "create",
        "tag": "guitar",
        "task": "loop %note%",
        "steps": [
            "identify the note in each string",
            "start the metronome at 30",
            "loop over each string 3 times"
        ]
    }

    add_task(task)

    # Example usage of add_placeholders
    placeholders = {
        "note": ["C", "C#", "D"],
        "color": ["red", "blue", "green"]
    }

    add_placeholders(placeholders)

    download_database()

Choose an option:
1. Create empty database
2. Create database with example data
3. Upload database
2


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# @title Rank Columns
import sqlite3
import functools

def ask_user_cmp(item1, item2):
    while True:
        print(f" [ 1 ] [{item1}] ?" )
        print(f" [ 2 ] [{item2}] ?")
        cmp = input(" --> ? ")
        if cmp == "1":
            return 1
        if cmp == "2":
            return -1
        print("1 or 2, please!")

##############################################################
def ask_user_cmp(item1, item2):
    # This function sorts items alphabetically in ascending order for automated testing.
    # To switch back to manual sorting, delete this function and use the original 'ask_user_cmp'.
    return (item1 < item2) - (item1 > item2)
##############################################################

def check_minimum_tasks(database_path):
    my_connection = sqlite3.connect(database_path)
    my_cursor = my_connection.cursor()
    my_cursor.execute("""
        SELECT P.ID, COUNT(T.ID)
        FROM PLACES P
        JOIN CONTEXTS C ON P.ID = C.PLACE_ID
        JOIN TAGS TG ON C.ID = TG.CONTEXT_ID
        JOIN TASKS T ON TG.ID = T.TAG_ID
        GROUP BY P.ID
    """)
    task_counts = my_cursor.fetchall()
    my_connection.close()
    for place_id, count in task_counts:
        if count < 5:
            raise ValueError(f"Place ID {place_id} has only {count} tasks, which is less than the required minimum of 5.")

def fetch_records_for_ranking(database_path, table_name, parent_column=None, rank_column='RANK'):
    my_connection = sqlite3.connect(database_path)
    my_cursor = my_connection.cursor()
    my_cursor.execute(f"""
        SELECT * FROM {table_name}
        WHERE {parent_column} IN (
            SELECT {parent_column} FROM {table_name} WHERE {rank_column} IS NULL
        ) AND {rank_column} IS NULL
        ORDER BY {parent_column}, {rank_column} ASC
    """)
    records = my_cursor.fetchall()
    my_connection.close()
    return records

def update_rank(database_path, table_name, record_id, new_rank, rank_column='RANK'):
    my_connection = sqlite3.connect(database_path)
    my_cursor = my_connection.cursor()
    my_cursor.execute(f"UPDATE {table_name} SET {rank_column} = {new_rank} WHERE ID = {record_id}")
    my_connection.commit()
    my_connection.close()

def sort_and_update(database_path, table_name, parent_column=None, rank_column='RANK'):
    records_to_rank = fetch_records_for_ranking(database_path, table_name, parent_column, rank_column)
    grouped_records = {}
    for record in records_to_rank:
        parent_id = record[1] if parent_column else 'default'
        if parent_id not in grouped_records:
            grouped_records[parent_id] = []
        grouped_records[parent_id].append(record)

    ask_user_key = functools.cmp_to_key(lambda x, y: ask_user_cmp(x[2], y[2]))  # Comparing by name

    print(f"Sorting for table: {table_name}")
    for group, records in grouped_records.items():
        print(f"Sorting group: {group}")
        records.sort(key=ask_user_key, reverse=True)
        for rank, record in enumerate(records, start=1):
            update_rank(database_path, table_name, record[0], rank, rank_column)

if __name__ == "__main__":
    db_path = '/content/tasks.db'  # Replace with your database path

    # Sort and update all necessary rankings in order, skipping PLACES and handling STEPS separately
    for table_name, parent_column in [('CONTEXTS', 'PLACE_ID'), ('TAGS', 'CONTEXT_ID'),
                                      ('TASKS', 'TAG_ID'), ('PLACEHOLDERS', 'TYPE')]:
        sort_and_update(db_path, table_name, parent_column)


Sorting for table: CONTEXTS
Sorting group: 1
Sorting group: 2
Sorting group: 3
Sorting for table: TAGS
Sorting group: 1
Sorting group: 2
Sorting group: 3
Sorting group: 4
Sorting group: 5
Sorting group: 6
Sorting group: 7
Sorting for table: TASKS
Sorting group: 1
Sorting group: 2
Sorting group: 3
Sorting group: 4
Sorting group: 5
Sorting group: 6
Sorting group: 7
Sorting group: 8
Sorting group: 9
Sorting group: 10
Sorting group: 11
Sorting group: 12
Sorting group: 13
Sorting group: 14
Sorting group: 15
Sorting group: 16
Sorting group: 17
Sorting group: 18
Sorting for table: PLACEHOLDERS
Sorting group: book
Sorting group: color
Sorting group: duration
Sorting group: equipment
Sorting group: event
Sorting group: flavor
Sorting group: herb
Sorting group: instrument
Sorting group: item
Sorting group: material
Sorting group: note
Sorting group: period
Sorting group: plant
Sorting group: time
Sorting group: topic
